In [63]:
#https://keras.io/examples/timeseries/timeseries_transformer_classification/
#https://saturncloud.io/blog/understanding-keras-conv1d-layer-parameters-filters-and-kernel-size/#:~:text=Filters%20in%20Conv1D&text=In%20the%20context%20of%20Conv1D,pattern%20in%20the%20input%20data.
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import plotly.graph_objects as go

df = pd.read_excel('Data.xlsx', sheet_name=6)
ordered_df = df.iloc[::-1]

date_list = df.iloc[:, 0].values
price_list = df.iloc[:, 1].values
features_list = df.iloc[:, 1:16].values

#Scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features_list)
price_list = np.reshape(price_list, (price_list.shape[0], 1))
output_scaler = StandardScaler()
scaled_output_data = output_scaler.fit_transform(price_list)
print(scaled_output_data, scaled_data)
#Train test split
training_data_len = math.ceil(len(scaled_data)* .8)
training_data_len

#Train test split
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
look_back_window = 12

for i in range(look_back_window, len(train_data)):
    x_train.append(train_data[i-look_back_window:i])
    y_train.append(train_data[i, 0])
#Convert to np array
x_train, y_train = np.array(x_train), np.array(y_train)

test_data = scaled_data[training_data_len-look_back_window:, :]

x_test = []
y_test = price_list[training_data_len:]
y_test = np.array(y_test)
y_test = np.reshape(y_test, (y_test.shape[0], 1))

for i in range(look_back_window, len(test_data)):
    x_test.append(test_data[i-look_back_window:i])

x_test = np.array(x_test)

[[-0.46940272]
 [-0.52389849]
 [-0.52127137]
 ...
 [ 2.37696601]
 [ 2.40632341]
 [ 2.39438903]] [[-0.46940272  0.13274505 -0.45581854 ... -1.12988853 -0.434165
  -0.57681784]
 [-0.52389849  0.2997445  -0.47795805 ... -1.09742737  0.66558632
   0.09290279]
 [-0.52127137  0.73211242 -0.51872857 ... -1.09670065  0.66643686
   0.08928268]
 ...
 [ 2.37696601 -0.64178697  2.36635699 ...  3.62238733 -0.66976524
  -0.59672845]
 [ 2.40632341 -0.89322437  2.39136356 ...  2.4601506  -0.52687412
  -0.56143236]
 [ 2.39438903 -0.99259071  2.40209315 ...  2.3247775  -0.55834419
  -0.56324242]]


In [64]:
from tensorflow import keras
from tensorflow.keras import layers
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [ ]:
input_shape = x_train.shape[1:]

head_size_list = [64, 128, 512]
ff_dim_list = [2, 4, 8, 16]
mlp_units_list = [64, 128, 512]
mlp_dropout_list = [0.1, 0.2, 0.3]
dropout_list = [0.1, 0.2, 0.3]
result = []
for head_size in head_size_list:
    for ff_dim in ff_dim_list:
        for mlp_units in mlp_units_list:
            for mlp_dropout in mlp_dropout_list:
                for dropout in dropout_list:
                    model = build_model(
                        input_shape,
                        head_size=head_size,
                        num_heads=4,
                        ff_dim=ff_dim,
                        num_transformer_blocks=10,
                        mlp_units=[mlp_units],
                        mlp_dropout=mlp_dropout,
                        dropout=dropout,
                    )

                    model.compile(optimizer='adam', loss="mean_squared_error")

                    callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

                    model.fit(
                        x_train,
                        y_train,
                        validation_split=0.2,
                        epochs=200,
                        batch_size=64,
                        callbacks=callbacks,
                        verbose=0
                    )
                    predictions = model.predict(x_test)
                    predictions = output_scaler.inverse_transform(predictions)
                    rmse = np.sqrt(np.mean(predictions - y_test)**2)
                    print(head_size, ff_dim, mlp_units, mlp_dropout, dropout)
                    print(rmse)
                    result.append({"rmse":rmse, "head_size":head_size, "ff_dim":ff_dim, "mlp_units":mlp_units, "mlp_dropout":mlp_dropout, "dropout":dropout})
                    if  rmse < 100:
                        train = np.squeeze(price_list[:training_data_len])
                        valid = np.squeeze(price_list[training_data_len:])
                        predictions = np.squeeze(predictions)
                        train_date_list = date_list[:training_data_len]
                        valid_date_list = date_list[training_data_len:]
                        valid_date_list2 = date_list[training_data_len:]
                        fig = go.Figure()
                        fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
                        fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
                        fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
                        fig.show()


In [37]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=8,
    mlp_units=[128],
    mlp_dropout=0.25,
    dropout=0.25,
)
model.summary()
model.compile(optimizer='adam', loss="mean_squared_error")

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)
predictions = model.predict(x_test)
predictions = output_scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)

Model: "model_291"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_292 (InputLayer)         [(None, 14, 14)]     0           []                               
                                                                                                  
 layer_normalization_5824 (Laye  (None, 14, 14)      28          ['input_292[0][0]']              
 rNormalization)                                                                                  
                                                                                                  
 multi_head_attention_2912 (Mul  (None, 14, 14)      60430       ['layer_normalization_5824[0][0]'
 tiHeadAttention)                                                , 'layer_normalization_5824[0][0]
                                                                 ']                       

In [38]:
predictions = model.predict(x_test)
predictions = output_scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)

8/8 [==============================] - 1s 151ms/step
28.96345651518039


In [39]:
train = np.squeeze(price_list[:training_data_len])
valid = np.squeeze(price_list[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
valid_date_list2 = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
fig.show()

In [65]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=8,
    num_transformer_blocks=10,
    mlp_units=[128],
    mlp_dropout=0.2,
    dropout=0.2,
)
model.summary()
model.compile(optimizer='adam', loss="mean_squared_error")

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
    shuffle=True
)
predictions = model.predict(x_test)
predictions = output_scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(rmse)

Model: "model_299"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_303 (InputLayer)         [(None, 12, 13)]     0           []                               
                                                                                                  
 layer_normalization_5960 (Laye  (None, 12, 13)      26          ['input_303[0][0]']              
 rNormalization)                                                                                  
                                                                                                  
 multi_head_attention_2980 (Mul  (None, 12, 13)      56333       ['layer_normalization_5960[0][0]'
 tiHeadAttention)                                                , 'layer_normalization_5960[0][0]
                                                                 ']                       

In [66]:
train = np.squeeze(price_list[:training_data_len])
valid = np.squeeze(price_list[training_data_len:])
predictions = np.squeeze(predictions)
train_date_list = date_list[:training_data_len]
valid_date_list = date_list[training_data_len:]
valid_date_list2 = date_list[training_data_len:]
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_date_list, y=train, name='train',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list, y=valid, name='valid',  mode='lines'))
fig.add_trace(go.Scatter(x=valid_date_list2, y=predictions, name='predict',  mode='lines'))
fig.show()